In [1]:
# Lots of imports, coppied from astroquery documentation
import sys
import os
import time
import re
import json

try: # Python 3.x
    from urllib.parse import quote as urlencode
    from urllib.request import urlretrieve
except ImportError:  # Python 2.x
    from urllib import pathname2url as urlencode
    from urllib import urlretrieve

try: # Python 3.x
    import http.client as httplib 
except ImportError:  # Python 2.x
    import httplib   

from astropy.table import Table
import numpy as np

import pprint
pp = pprint.PrettyPrinter(indent=4)

In [2]:
#Advanced mast query builder. Based on code from astroquery documentation

def mastQuery(request):
    """Perform a MAST query.
    
        Parameters
        ----------
        request (dictionary): The MAST request json object
        
        Returns head,content where head is the response HTTP headers, and content is the returned data"""
    
    server='mast.stsci.edu'

    # Grab Python Version 
    version = ".".join(map(str, sys.version_info[:3]))

    # Create Http Header Variables
    headers = {"Content-type": "application/x-www-form-urlencoded",
               "Accept": "text/plain",
               "User-agent":"python-requests/"+version}

    # Encoding the request as a json string
    requestString = json.dumps(request)
    requestString = urlencode(requestString)
    
    # opening the https connection
    conn = httplib.HTTPSConnection(server)

    # Making the query
    conn.request("POST", "/api/v0/invoke", "request="+requestString, headers)

    # Getting the response
    resp = conn.getresponse()
    head = resp.getheaders()
    content = resp.read().decode('utf-8')

    # Close the https connection
    conn.close()

    return head,content

In [5]:
# Test 1: Basic name lookup

objectOfInterest = 'M101'

resolverRequest = {'service':'Mast.Name.Lookup',
                     'params':{'input':objectOfInterest,
                               'format':'json'},
                     }

headers,resolvedObjectString = mastQuery(resolverRequest)

resolvedObject = json.loads(resolvedObjectString)

pp.pprint(resolvedObject)

{   'resolvedCoordinate': [   {   'cacheDate': 'Jun 13, 2019 5:52:09 PM',
                                  'cached': True,
                                  'canonicalName': 'MESSIER 101',
                                  'decl': 54.34895,
                                  'objectType': 'G',
                                  'ra': 210.80227,
                                  'radius': 0.24000000000000002,
                                  'resolver': 'NED',
                                  'resolverTime': 233,
                                  'searchRadius': -1.0,
                                  'searchString': 'm101'}],
    'status': ''}
[('Server', 'nginx/1.12.2'), ('Date', 'Wed, 26 Jun 2019 19:30:49 GMT'), ('Content-Type', 'application/json;charset=UTF-8'), ('Transfer-Encoding', 'chunked'), ('Connection', 'keep-alive'), ('Cache-Control', 'private, max-age=0'), ('X-AspNet-Version', '4.0.30319'), ('Set-Cookie', 'MAST_Session=Id=ced8b6f5-66a3-439e-bf3c-ecb63368c3c1; path=/'), ('X

In [4]:
# Get right ascnesion and declination
objRa = resolvedObject['resolvedCoordinate'][0]['ra']
objDec = resolvedObject['resolvedCoordinate'][0]['decl']

In [6]:
mashupRequest = {"service":"Mast.Caom.Filtered",
                 "format":"json",
                 "params":{
                     "columns":"COUNT_BIG(*)",
                     "filters":[
                         {"paramName":"filters",
                          "values":["NUV","FUV"],
                          "separator":";"
                         },
                         {"paramName":"t_max",
                          "values":[{"min":52264.4586,"max":54452.8914}], #MJD
                         },
                         {"paramName":"obsid",
                          "values":[],
                          "freeText":"%200%"}
                     ]}}
    
headers,outString = mastQuery(mashupRequest)
countData = json.loads(outString)

In [8]:
countData

{'status': 'COMPLETE',
 'msg': '',
 'data': [{'Column1': 1068}],
 'fields': [{'name': 'Column1', 'type': 'string'}],
 'paging': {'page': 1,
  'pageSize': 1,
  'pagesFiltered': 1,
  'rows': 1,
  'rowsFiltered': 1,
  'rowsTotal': 1}}

In [22]:
#build basic request. Will loop over stars, build dataframe
# I think this will negate having to do name lookups
requestTic = {"service":"Mast.Catalogs.Filtered.Tic",
              "format":"json",
              "params":{
                  "columns":"c.GAIA", #Column name to get
                  "filters":[
                       {"paramName":"ID",
                         "values":["101417386"]}]
                    }
}

headers,outString = mastQuery(requestTic)
outData = json.loads(outString)

In [23]:
#Test

outString

'{"status" : "COMPLETE","msg" : "","data":[{"GAIA":"5670584955144424576"}],"fields":[{"name":"GAIA","type":"string"}],"paging":{"page":1,"pageSize":1,"pagesFiltered":1,"rows":1,"rowsFiltered":1,"rowsTotal":1}}'

In [38]:
def aqTicReq(tessid, column):
    requestTic = {"service":"Mast.Catalogs.Filtered.Tic",
                  "format":"json",
                  "params":{
                      "columns":"c.{}".format(str(column)), #Column name to get
                      "filters":[
                           {"paramName":"ID",
                             "values":[str(tessid)]}]
                        }
    }

    headers,outString = mastQuery(requestTic)
    return(outData['data'][0][str(column)])

In [39]:
aqTicReq(118313102, ('GAIA'))

'5670584955144424576'

In [42]:
round(float(aqTicReq(118313102, ('ra')), 10)) 

KeyError: 'ra'